# FGGLIB Introduction

Factor graph grammars (FGG) are hyperedge replacement grammar that generate factor graphs.  <br>
They overcome the limitation that factor graphs have fixed structures, allow tractable inferences in many cases and can be useful to represent context free grammars - especially for probabilistic context free grammar which can generate more than one parse tree.  <br>
<br>
David Chiang et al. present factor graph grammars and algorithms for conjunction of and inference on them in their work Factor Graph Grammar.  <br>
As part of the course project of Advanced Formal Language Theory, we have implemented the algorithms in a library called FGGLIB. <br>
FGGLIB is a Python library structred in a same way as the paper and includes tests on a representative set of FGG examples.

***

### Installation

To install and use this library, download the source code and run _pip install -e ._ in the top directory.  <br>
In order to run all packages we recommend to use Python 3.10 and pip 21.


In [12]:
from IPython.display import Image

from fgglib.presentation.visualization import Visualization
from fgglib.fg.functions.circuit import *
from fgglib.fg.functions.discretedensity import DiscreteDensity



## Hypergraph

First, we will start with building a basic hypergraph that consists of labeled vertices and edges. <br>
As an example from <a href="https://upload.wikimedia.org/wikipedia/commons/thumb/2/2f/Directed_hypergraph_example.svg/330px-Directed_hypergraph_example.svg.png">Wikipedia</a> we create a hyperedge that has six vertices and five edges as provided below. The elements of vertices and edges can be anything. 

<img src="hypergraph.png" width="130"/> <br>

In [13]:
from fgglib.fg.hypergraph import Hypergraph
from fgglib.fg.vertex import Vertex
from fgglib.fg.edge import Edge

hypergraph = Hypergraph()
vertex1 = Vertex('V1','v1')
vertex2 = Vertex('V2','v2')
vertex3 = Vertex('V3','v3')
vertex4 = Vertex('V4','v4')
vertex5 = Vertex('V5','v5')
vertex6 = Vertex('V6','v6')

edge1 = Edge(('V1, V2'),'e1')
edge2 = Edge({'V2, V3'},'e2')
edge3 = Edge({'V1, V3'},'e3')
edge4 = Edge({'V2, V3, V4, V5'},'e4')
edge5 = Edge({'V3', 'V5', 'V6'},'e5')

hypergraph.add_vertex(vertex1)
hypergraph.add_vertex(vertex2)
hypergraph.add_vertex(vertex3)
hypergraph.add_vertex(vertex4)
hypergraph.add_vertex(vertex5)
hypergraph.add_vertex(vertex6)

hypergraph.add_edge(edge1)
hypergraph.add_edge(edge2)
hypergraph.add_edge(edge3)
hypergraph.add_edge(edge4)
hypergraph.add_edge(edge5)

edge1.add_target(vertex1)
edge1.add_target(vertex2)
edge2.add_target(vertex2)
edge2.add_target(vertex3)
edge3.add_target(vertex1)
edge3.add_target(vertex3)
edge4.add_target(vertex2)
edge4.add_target(vertex3)
edge4.add_target(vertex4)
edge4.add_target(vertex5)
edge5.add_target(vertex3)
edge5.add_target(vertex5)
edge5.add_target(vertex6)

print(hypergraph.V)
print(hypergraph.E)

{Vertex: v5, Vertex: v1, Vertex: v3, Vertex: v2, Vertex: v6, Vertex: v4}
{Edge e5: [Vertex: v3, Vertex: v5, Vertex: v6], Edge e3: [Vertex: v1, Vertex: v3], Edge e4: [Vertex: v2, Vertex: v3, Vertex: v4, Vertex: v5], Edge e1: [Vertex: v1, Vertex: v2], Edge e2: [Vertex: v2, Vertex: v3]}


## Factor Graph

Factor graphs are probabilistic models which can be described as hypergraphs. They consist of variables, which are identical properties of the graph nodes and factors, a property of the hyperedges. These factors can be associated with some function and used to calculate a probability for a given variable assignment. <br>
FGGLIB also enables usage of different semirings which is part of rayuela library. For simplicity, the same notation was used.<br>
<br>
Using FGGLIB an examplary factor graph from the original paper (Example3) can be generated, where vertices are drawn in circle and edges in rectangles connecting the vertices. <br> <br>
 <img src="fg_exp3.png" width="450"/> <br>

In [14]:
from fgglib.fg.factorgraph import Factorgraph
from fgglib.base.semiring import Real

fg = Factorgraph(Real)
vertexSet = ('T0','T1','W2','T3','W4','T5','W6','T7')

fg.createVertices(None, vertexSet, Real)

fg.createEdge('BOS','e1', {'T0'}, None, Real)
fg.createEdge('T1|T0','e2', {'T0','T1'}, None, Real)
fg.createEdge('W2|T1','e3', {'T1','W2'}, None, Real)
fg.createEdge('T3|T1','e4', {'T1','T3'}, None, Real)
fg.createEdge('W4|T3','e5', {'T3','W4'}, None, Real)
fg.createEdge('T5|T3','e6', {'T3','T5'}, None, Real)
fg.createEdge('W6|T5','e7', {'T5','W6'}, None, Real)
fg.createEdge('T7|T5','e8', {'T5','T7'}, None, Real)
fg.set_function(fg.get_edge('e1'), DiscreteDensity([[3, 4],[3,9]]))
fg.set_function(fg.get_edge('e2'), DiscreteDensity([[6, 7],[7,6]]))
fg.set_function(fg.get_edge('e3'), DiscreteDensity([[3, 4],[5,1]]))
fg.set_function(fg.get_edge('e4'), DiscreteDensity([[7, 8],[3,7]]))
fg.set_function(fg.get_edge('e5'), DiscreteDensity([[5, 2],[4,9]]))
fg.set_function(fg.get_edge('e6'), DiscreteDensity([[8, 1],[3,1]]))
fg.set_function(fg.get_edge('e7'), DiscreteDensity([[1, 1],[6,8]]))
fg.set_function(fg.get_edge('e8'), DiscreteDensity([[2, 9],[4,3]]))


Visualization().draw(fg)

 <img src="../../Graph.png" width="200"/> <br>

***
A factor graph has a function F that maps edge labels to functions. <br>
FGGLIB provides three different function types: circuit, discrete density and normal density functions. <br>
Below we use discrete density with numerical values to draw a factor graph alculate marginals.

In [15]:
fg.set_function(fg.get_edge('e1'), DiscreteDensity([[3, 4],[3,9]]))
fg.set_function(fg.get_edge('e2'), DiscreteDensity([[6, 7],[7,6]]))
fg.set_function(fg.get_edge('e3'), DiscreteDensity([[3, 4],[5,1]]))
fg.set_function(fg.get_edge('e4'), DiscreteDensity([[7, 8],[3,7]]))
fg.set_function(fg.get_edge('e5'), DiscreteDensity([[5, 2],[4,9]]))
fg.set_function(fg.get_edge('e6'), DiscreteDensity([[8, 1],[3,1]]))
fg.set_function(fg.get_edge('e7'), DiscreteDensity([[1, 1],[6,8]]))
fg.set_function(fg.get_edge('e8'), DiscreteDensity([[2, 9],[4,3]]))

marginals = fg.sum_product()
print(marginals[fg.get_vertex('T1')].normalize().pmf)

[0.26834413 0.73165587]


## Factor Graph Grammars

Factor Graph Grammar (FGG) is a hyperedge replacement graph grammar that generates factor graphs. Hyperedge replacement grammars behave very similarly to context-free grammars, but their productions replace hyperedges of the graph with specific subgraphs. This way, a graph of any arbitrary size can be constructed. <br>
FGG is a 4-Tuple <T,N,S,P> where:
- N is a finite set of nonterminal symbols
- T is a finite set of terminal symbols (corresponds to factor graph fragments in our library)
- P is a finite set of production rules of the form (X -> R), where X is an element of nonterminal N and R is a factor graph fragment with edge labels in N ∪ T. 
- S is a string (or label) for the starting nonterminal.

Let us create an FGG for derivations of a CFG in Chomsky Normal Form which was presented in lecture note 6.
- Nominal → Det NP
- NP → Adj NP
- Det → the
- N → male
- Adj → big

Every right-hand side of production rule is a factor graph with the corresponding edge labels.




In [16]:
from fgglib.fgg.fgg import FGG
from fgglib.fgg.production import Production
from fgglib.fg.fragment import Fragment
frag=Fragment()

nominal = frag.buildFragment(
    {'NOM','DET', 'NP'}, # V
    [('NOMINAL_', {'NOM','DET', 'NP'})], # E
    {'DET', 'NP'}, # ext
)

determinant = frag.buildFragment(
    {'DET', 'the'}, # V
    [('DET_', {'DET','the'})], # E
    {'the'}, # ext
)

nounphrase = frag.buildFragment(
    {'NP', 'ADJ', 'N'}, # V
    [('NP_', {'NP', 'ADJ','N'})], # E
    {'ADJ','NP'}, # ext
)

adj = frag.buildFragment(
     {'ADJ', 'big'}, # V
    [('ADJ_', {'ADJ','big'})], # E
    {'big'}, # ext
)

noun = frag.buildFragment(
    {'N', 'man'}, # V
    [('N_', {'N','man'})], # E
    {'man'}, # ext
)

derivationtree = FGG({nominal,determinant, nounphrase, adj, noun}, # T
    {'NOMINAL','DET', 'NP', 'ADJ', 'N'}, # N
    {'NOMINAL'}, # S
    {Production('S', nominal),
     Production('Det', determinant),
     Production('NP', nounphrase),
     Production('Adj', adj),
     Production('N', noun)} # P
)

Visualization().draw(derivationtree)

 <img src="../../FGG.png" width="200"/> <br>

## Conjunction

Conjunction is used to calculate the marginal distribution of a subset of nodes without having to identify the node in the grammar explicitly, instead it uses the structure of the graph. <br>
Using the example of "the big male" from the previous part: "derivationtree" and "noun" are conjoined in order to calculate the marginal distribution of the noun "male". As a result of conjunction a new FGG is returned with conjoined pairs of rules.

In [17]:
#We create noun factor graph grammar from the fraction
#declared in the previous example.

nounFGG = FGG(
    {noun}, # T
    {'X'}, # N
    'S', # S
    {Production('X',noun)} # P
)

conjoined = derivationtree.conjunction(nounFGG)

Visualization().draw(conjoined)
print(conjoined)


start: {'NOMINAL'}
('N', 'X') → [Graph: {Vertex: N, Vertex: man} | {Edge N_: [Vertex: N, Vertex: man]}]



 <img src="../../FGG.png" width="200"/> <br>

***
## Inference